In [ ]:
!pip install fake-useragent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from bs4 import BeautifulSoup
import requests
import os
import time
import pandas as pd

In [ ]:
City = 'mumbai'

In [ ]:
data_dir = '/content/drive/MyDrive/DsLabMp/'

subdirectories = ['Data', f'Data/{City}', f'Data/{City}/Flats', f'Data/{City}/Societies', f'Data/{City}/Residential', f'Data/{City}/Independent House']

for subdir in subdirectories:
  dir = os.path.join(data_dir,subdir)

  if not os.path.exists(dir):
    os.makedirs(dir)
    print(f'Created Directory {dir}')
  else:
    print(f'Directory {dir} already exist !')



Directory /content/drive/MyDrive/DsLabMp/Data already exist !
Directory /content/drive/MyDrive/DsLabMp/Data/mumbai already exist !
Directory /content/drive/MyDrive/DsLabMp/Data/mumbai/Flats already exist !
Directory /content/drive/MyDrive/DsLabMp/Data/mumbai/Societies already exist !
Directory /content/drive/MyDrive/DsLabMp/Data/mumbai/Residential already exist !
Directory /content/drive/MyDrive/DsLabMp/Data/mumbai/Independent House already exist !


In [ ]:
pheaders = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'en-US,en;q=0.9,hi;q=0.8,mr;q=0.7,bho;q=0.6',
    'Content-Type': 'text/plain',
    'Cookie': 'ar_debug=1',  # Keep or remove based on need
    'Origin': 'https://www.99acres.com',
    'Referer': f'https://www.99acres.com/resale-flats-in-{City}-ffid-page',  # Update referer
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-Storage-Access': 'active',
    'Sec-CH-UA': '"Not(A:Brand";v="99", "Google Chrome";v="133", "Chromium";v="133"',
    'Sec-CH-UA-Mobile': '?0',
    'Sec-CH-UA-Platform': '"Windows"',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36',
    'Priority': 'u=1, i',
    'Authority': 'www.google-analytics.com',
    'Method': 'POST',  # You can change it to GET if you don't need to send POST data
    'Scheme': 'https'
}

cheaders = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'en-US,en;q=0.9,hi;q=0.8,mr;q=0.7,bho;q=0.6',
    'Content-Type': 'text/plain',
    'Cookie': 'ar_debug=1',
    'Origin': 'https://www.99acres.com',
    'Referer': 'https://www.99acres.com/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-Storage-Access': 'active',
    'Sec-CH-UA': '"Not(A:Brand";v="99", "Google Chrome";v="133", "Chromium";v="133"',
    'Sec-CH-UA-Mobile': '?0',
    'Sec-CH-UA-Platform': '"Windows"',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36',
    'Priority': 'u=1, i',
    'Authority': 'www.google-analytics.com',
    'Method': 'POST',
    'Scheme': 'https'
}


In [ ]:

start = int(input('Enter starting page number :'))
end = start + 1

pageNum = start

flats = pd.DataFrame()

req=0

try:
  while pageNum < end:
    i=1
    url = f'https://www.99acres.com/resale-flats-in-{City}-ffid-page-{pageNum}'
    page = requests.get(url,headers = pheaders)
    pageSoup = BeautifulSoup(page.content, 'html.parser')
    req+=1 #counting each request then stoping for few seconds so server doesnt detect the bot

    for soup in pageSoup.select_one('div[data-label="SEARCH"]').select('section[data-hydration-on-demand="true"]'):
      try:
        link = soup.select('a.ellipsis')[0]['href'] #propertyLink
        property_name = soup.select('h2')[0].text #propertyname
        #society = soup.select('a.ellipsis')[0].text #SocietyName
        society = soup.find_all('div', {'class': 'tupleNew__locationName ellipsis'})[0].text.strip()
      except:
        continue

      proppage = requests.get(link, headers=cheaders)
      proppageSoup = BeautifulSoup(proppage.content, 'html.parser')
      req += 1

      try:
          #price Range
          price = proppageSoup.select_one('#pdPrice2').text.strip()
      except:
          price = ''

      # Area
      try:
          area = area = '₹' + proppageSoup.select_one('#pricePerUnitArea').text.strip().strip('@')
      except:
          area =''
      # Area with Type
      try:
          areaWithType = 'Carpet area: '+ proppageSoup.select_one('#carpetArea_span').text.strip() +' '+ proppageSoup.select_one('#displayBuiltupArea_span').text.strip()
      except:
          try:
              areaWithType = proppageSoup.select_one('#factArea').text.strip()
          except:
              areaWithType = ''


      # Configuration
      try:
          bedRoom = proppageSoup.select_one('#bedRoomNum').text.strip()
      except:
          bedRoom = ''
      try:
          bathroom = proppageSoup.select_one('#bathroomNum').text.strip()
      except:
          bathroom = ''
      try:
          balcony = proppageSoup.select_one('#balconyNum').text.strip()
      except:
          balcony = ''

      try:
          additionalRoom = proppageSoup.select_one('#additionalRooms').text.strip()
      except:
          additionalRoom = ''

      # Address

      try:
          address = proppageSoup.select_one('#address').text.strip()
      except:
          address = ''
      # Floor Number
      try:
          floorNum = proppageSoup.select_one('#floorNumLabel').text.strip()
      except:
          floorNum = ''

      try:
          facing = proppageSoup.select_one('#facingLabel').text.strip()
      except:
          facing = ''

      try:
          agePossession = proppageSoup.select_one('#agePossessionLbl').text.strip()
      except:
          agePossession = ''

      # Nearby Locations

      try:
          nearbyLocations = [i.text.strip() for i in proppageSoup.select_one('div.NearByLocation__tagWrap').select('span.NearByLocation__infoText')]
      except:
          nearbyLocations = ''

      # Descriptions
      try:
          description = proppageSoup.select_one('#description').text.strip()
      except:
          description = ''

      # Furnish Details
      try:
          furnishDetails = [i.text.strip() for i in proppageSoup.select_one('#FurnishDetails').select('li')]
      except:
          furnishDetails = ''

      # Features
      if furnishDetails:
          try:
              features = [i.text.strip() for i in proppageSoup.select('#features')[1].select('li')]
          except:
              features = ''
      else:
          try:
              features = [i.text.strip() for i in proppageSoup.select('#features')[0].select('li')]
          except:
              features = ''



      # Rating by Features
      try:
          rating = [i.text for i in proppageSoup.select_one('div.review__rightSide>div>ul>li>div').select('div.ratingByFeature__circleWrap')]
      except:
          rating = ''

      try:
          # Property ID
          property_id = proppageSoup.select_one('#Prop_Id').text.strip()
      except:
          property_id = ''

      property_data = {
      'property_name': property_name,
      'society': society,
      'link':link,
      'price': price,
      'area': area,
      'areaWithType': areaWithType,
      'bedRoom': bedRoom,
      'bathroom': bathroom,
      'balcony': balcony,
      'additionalRoom': additionalRoom,
      'address': address,
      'floorNum': floorNum,
      'facing': facing,
      'agePossession': agePossession,
      'nearbyLocations': nearbyLocations,
      'description': description,
      'furnishDetails': furnishDetails,
      'features': features,
      'rating': rating,
      'property_id': property_id
      }

      temp_df = pd.DataFrame.from_records([property_data])
      flats = pd.concat([flats, temp_df], ignore_index=True)
      i += 1
      if req % 4==0:
          time.sleep(10)
      if req % 15 == 0:
          time.sleep(50)

    print(f'{pageNum} -> {i}')
    pageNum += 1

  csv_file_path = f"/content/drive/MyDrive/DsLabMp/Data/mumbai/Flats/flats_{City}_data-page-{start}-{pageNum-1}.csv"

    # This file will be new every time if start page will chnage, but still taking here mode as append
  if os.path.isfile(csv_file_path):
    # Append DataFrame to the existing file without header
    flats.to_csv(csv_file_path, mode='a', header=False, index=False)
  else:
        # Write DataFrame to the file with header - first time write
    flats.to_csv(csv_file_path, mode='a', header=True, index=False)



except AttributeError as e:
  print(e)
  print("----------------")
  print("""Your IP might have blocked. Delete Runitme and reconnect again with updating start page number.\n
            You would see in output above like 1 -> 15\ and so 1 is page number and 15 is data items extracted.""")

csv_file_path = f"/content/drive/MyDrive/DsLabMp/Data/mumbai/Flats/flats_{City}_data-page-{start}-{pageNum-1}.csv"

    # This file will be new every time if start page will chnage, but still taking here mode as append
if os.path.isfile(csv_file_path):
  # Append DataFrame to the existing file without header
  flats.to_csv(csv_file_path, mode='a', header=False, index=False)
else:
  # Write DataFrame to the file with header - first time write
  flats.to_csv(csv_file_path, mode='a', header=True, index=False)

Enter starting page number :75


KeyboardInterrupt: 

In [ ]:
flats = flats.iloc[:83,:]

In [ ]:
csv_file_path = f"/content/drive/MyDrive/DsLabMp/Data/mumbai/Flats/flats_{City}_data-page-{start}-{pageNum-1}.csv"

    # This file will be new every time if start page will chnage, but still taking here mode as append
if os.path.isfile(csv_file_path):
  # Append DataFrame to the existing file without header
  os.remove(csv_file_path)

flats.to_csv(csv_file_path, mode='a', header=True, index=False)

##debugging

In [ ]:
proppage = requests.get('https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-sale-in-veera-desai-road-mumbai-andheri-dahisar-750-sq-ft-spid-S79968421', headers=cheaders)
proppageSoup = BeautifulSoup(proppage.content, 'html.parser')

In [ ]:
area = '₹' + proppageSoup.select_one('#pricePerUnitArea').text.strip().strip('@')

In [ ]:
area

'₹ 26,666 per sq.ft.'